# HW5 - Matin Mahmoodkhani - 99522095

Getting Data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller

In [2]:
import requests

def get_top_100_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        coins = [coin['symbol'].upper() + "-USD" for coin in data]
        return coins
    else:
        print("Error:", response.status_code)
        return None

symbols = get_top_100_coins()
print(symbols)


['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'SOL-USD', 'XRP-USD', 'USDC-USD', 'STETH-USD', 'ADA-USD', 'AVAX-USD', 'DOGE-USD', 'DOT-USD', 'TRX-USD', 'LINK-USD', 'MATIC-USD', 'TON-USD', 'WBTC-USD', 'ICP-USD', 'SHIB-USD', 'DAI-USD', 'LTC-USD', 'BCH-USD', 'UNI-USD', 'ATOM-USD', 'LEO-USD', 'OP-USD', 'LDO-USD', 'INJ-USD', 'NEAR-USD', 'XLM-USD', 'OKB-USD', 'ETC-USD', 'FIL-USD', 'XMR-USD', 'STX-USD', 'HBAR-USD', 'IMX-USD', 'APT-USD', 'ARB-USD', 'KAS-USD', 'CRO-USD', 'TIA-USD', 'VET-USD', 'MNT-USD', 'TUSD-USD', 'FDUSD-USD', 'BSV-USD', 'MKR-USD', 'QNT-USD', 'SEI-USD']


In [3]:
import yfinance as yf
# import datetime

start = "2022-11-01"
end = "2023-11-01"


dfs = {}

for symbol in symbols:
    ticker = yf.Ticker(symbol)
    data = ticker.history(interval="1h", start=start, end=end)
    if len(data.dropna()) >= 10:
        resampled_df = data.resample('4h').agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        })
        dfs[symbol] = resampled_df['Close']

IMX-USD: No price data found, symbol may be delisted (1h 2022-11-01 -> 2023-11-01)


In [4]:
dfs

{'BTC-USD': Datetime
 2022-11-01 00:00:00+00:00    20496.050781
 2022-11-01 04:00:00+00:00    20585.644531
 2022-11-01 08:00:00+00:00    20528.517578
 2022-11-01 12:00:00+00:00    20423.361328
 2022-11-01 16:00:00+00:00    20447.882812
                                  ...     
 2023-10-31 04:00:00+00:00    34153.902344
 2023-10-31 08:00:00+00:00    34518.582031
 2023-10-31 12:00:00+00:00    34290.898438
 2023-10-31 16:00:00+00:00    34493.507812
 2023-10-31 20:00:00+00:00    34668.410156
 Freq: 4H, Name: Close, Length: 2190, dtype: float64,
 'ETH-USD': Datetime
 2022-11-01 00:00:00+00:00    1586.926025
 2022-11-01 04:00:00+00:00    1589.435059
 2022-11-01 08:00:00+00:00    1590.957397
 2022-11-01 12:00:00+00:00    1572.205200
 2022-11-01 16:00:00+00:00    1577.353760
                                 ...     
 2023-10-31 04:00:00+00:00    1791.078735
 2023-10-31 08:00:00+00:00    1810.569580
 2023-10-31 12:00:00+00:00    1794.058105
 2023-10-31 16:00:00+00:00    1809.860474
 2023-10-31

Part 1

In [5]:
def adf_test(series):
    result = adfuller(series)
#     print('ADF Statistic: %f' % result[0])
#     print('p-value: %f' % result[1])
    return result[1]

In [18]:
# Source : https://blog.quantinsti.com/johansen-test-cointegration-building-stationary-portfolio/

from statsmodels.tsa.vector_ar.vecm import coint_johansen
import pandas as pd


def johansen_test(symbol1, symbol2):
    dfs[symbol1] = dfs[symbol1].dropna()
    dfs[symbol2] = dfs[symbol2].dropna()
    data = pd.concat([dfs[symbol1],dfs[symbol2]], axis=1)
    try:
        result = coint_johansen(data.values, det_order=0, k_ar_diff=1)
        weights = result.evec
        newSeries = 0

        for i in range(2):
            newSeries += np.dot(data.values, weights[:, i])
        adf_result = adf_test(newSeries)
        if adf_result <= 0.05:
            return weights,adf_result,newSeries
        else:
            return None, None, None
    except:
        return None, None,None

In [19]:
syms = dfs.keys()
syms1 = list(syms)

In [21]:
timeSeries = []
counter = 0
for index1 in range(len(dfs.keys())):
    if(counter==10):
            break
    for index2 in range(index1+1,len(dfs.keys())):
        if(counter==10):
            break
        a = syms1[index1]
        b = syms1[index2]
        result = johansen_test(a, b)
        if result[1]:
            timeSeries.append([a, b, result[0], result[1], result[2]])
            counter += 1

In [27]:
for i in timeSeries:
    print(f"Coins are {i[0]} & {i[1]}")
    print(f"P-Value of the TimeSeries: {i[3]}")
    print(f"Weights are: {i[2]}")
    print(f"Time Series: {i[4]}")
    print()

Coins are BTC-USD & USDT-USD
P-Value of the TimeSeries: 0.03849275182272709
Weights are: [[ 1.62682064e-05 -2.11630763e-04]
 [-9.75226841e+02  1.96017466e+01]]
Time Series: [-959.4925518  -959.48630287 -959.49627445 ... -962.84976191 -962.56786368
 -962.74329309]

Coins are BTC-USD & USDC-USD
P-Value of the TimeSeries: 0.0006366988882504506
Weights are: [[ 1.13398749e-05 -2.11256726e-04]
 [-2.61602203e+02  6.31426013e-03]]
Time Series: [-265.675448   -265.73403969 -265.67649694 ... -268.47039583 -268.48511115
 -268.53682324]

Coins are BTC-USD & DOGE-USD
P-Value of the TimeSeries: 0.008032703364961567
Weights are: [[7.37689607e-05 2.23940712e-04]
 [8.59199353e+01 1.22532196e+01]]
Time Series: [18.62420137 20.37223534 20.08195657 ... 16.81140477 16.95310288
 17.0227248 ]

Coins are BTC-USD & WBTC-USD
P-Value of the TimeSeries: 0.01097840881614605
Weights are: [[ 3.38278792e-03 -2.06627000e-04]
 [-3.39429518e-03 -4.64551193e-06]]
Time Series: [-4.62142457 -4.56974759 -4.58270543 ... -7.5

Part 2 & 3

In [33]:
# Source : https://stackoverflow.com/questions/39488806/hurst-exponent-in-python
from datetime import datetime
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn

def Hurst(ts):

    """Returns the Hurst Exponent of the time series vector ts"""
    lags = range(2, 100)
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = polyfit(log(lags), log(tau), 1)
    return poly[0]*2.0

In [55]:
# Source : https://www.pythonforfinance.net/2016/05/09/python-backtesting-mean-reversion-part-2/

import numpy as np
import statsmodels.api as sm

def calculate_half_life(time_series):

    # Create lagged series
    spread_lag = np.roll(time_series, 1)
    spread_lag[0] = spread_lag[1]

    # Calculate the return series
    spread_ret = time_series - spread_lag
    spread_ret[0] = spread_ret[1]

    # Add a constant to the lagged series
    spread_lag2 = sm.add_constant(spread_lag)

    # Fit an OLS regression model
    model = sm.OLS(spread_ret, spread_lag2)
    res = model.fit()

    # Calculate half-life
    half_life = -np.log(2) / res.params[1]

    return half_life

In [58]:
for i in timeSeries:
    hurstExpo = Hurst(i[4])
    print(f"Time Series of {i[0]} & {i[1]}")
    print(f"Hurst Exponent: {hurstExpo}")
    print(f"Half Time: {calculate_half_life(i[4])}\n")

Time Series of BTC-USD & USDT-USD
Hurst Exponent: 0.17490373742055246
Half Time: 5.912074705920939

Time Series of BTC-USD & USDC-USD
Hurst Exponent: 0.10244639430492913
Half Time: 6.393618952341948

Time Series of BTC-USD & DOGE-USD
Hurst Exponent: 0.4876787512264022
Half Time: 85.72296812230427

Time Series of BTC-USD & WBTC-USD
Hurst Exponent: 0.0993147928920039
Half Time: 2.553499715883438

Time Series of BTC-USD & DAI-USD
Hurst Exponent: 0.11140008682398017
Half Time: 5.831646854560554

Time Series of BTC-USD & STX-USD
Hurst Exponent: 0.05802520957100378
Half Time: 7.819571743917927

Time Series of BTC-USD & CRO-USD
Hurst Exponent: 0.5167168719246018
Half Time: 118.03731137000368

Time Series of BTC-USD & TUSD-USD
Hurst Exponent: 0.1196719760254706
Half Time: 5.240493787985892

Time Series of ETH-USD & USDT-USD
Hurst Exponent: 0.18006594099381654
Half Time: 5.628637364700789

Time Series of ETH-USD & USDC-USD
Hurst Exponent: 0.105716362140582
Half Time: 6.413911636188349

